In [ ]:
import os
import sys
from pathlib import Path

# Navigate to project root (equivalent to cd ..)
project_dir = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
os.chdir(project_dir)

# Add src directory to Python path for imports
src_dir = project_dir / "src"
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Set environment for dev testing
os.environ['REPORT_ENV'] = 'dev'

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import cdutils.acct_file_creation.core
from datetime import datetime

In [ ]:
df = cdutils.acct_file_creation.core.query_df_on_date()

In [ ]:
df

In [ ]:
# Fetch wh_org data
import src.non_profit_list.fetch_data
org_data = src.non_profit_list.fetch_data.fetch_data()

In [ ]:
import cdutils.deduplication

In [ ]:
# Dedupe org table
if 'wh_org' in org_data:
    dedupe_list = [{'df': org_data['wh_org'], 'field': 'orgnbr'}]
    org_data['wh_org'] = cdutils.deduplication.dedupe(dedupe_list)

In [ ]:
wh_org = org_data['wh_org'].copy()

In [ ]:
assert wh_org['orgnbr'].is_unique, "Not unique"

In [ ]:
wh_org

In [ ]:
INPUT_PATH_GENESIS = Path(r"C:\Users\w322800\Documents\gh\bcsb-prod\Reports\Community Relations\Non Profit List\input\Non-profit Organizations - Records - 8-18-2025.xlsx")
genesis_data = pd.read_excel(INPUT_PATH_GENESIS, header=1)

In [ ]:
genesis_data

In [ ]:
non_profits = wh_org.copy()

In [ ]:
non_profits.info()

In [ ]:
non_profits = non_profits[[
    'orgnbr',
    'orgname',
    'orgtypcd',
    'orgtypcddesc'
]].copy()

In [ ]:
# Aggregate stats (total loans/deposits) per orgnbr
## orgs only
acct_orgs = df[df['taxrptforpersnbr'].isna()].copy()

In [ ]:
import numpy as np

In [ ]:
## loans/deposit categorization
# Account type mappings
ACCOUNT_TYPE_MAPPING = {
    'CML': 'Commercial Loan',
    'MLN': 'Commercial Loan',
    'CNS': 'Consumer Loan',
    'MTG': 'Residential Loan',
    'CK': 'Checking',
    'SAV': 'Savings',
    'TD': 'CD'
}

acct_orgs['Account Type'] = acct_orgs['mjaccttypcd'].map(ACCOUNT_TYPE_MAPPING)

In [ ]:
acct_orgs = acct_orgs[~(acct_orgs['Account Type'].isna())].copy()

In [ ]:
acct_orgs

In [ ]:
MACRO_TYPE_MAPPING = {
    'CML': 'Loan',
    'MLN': 'Loan',
    'CNS': 'Loan',
    'MTG': 'Loan',
    'CK': 'Deposit',
    'SAV': 'Deposit',
    'TD': 'Deposit'
}

acct_orgs['Macro Account Type'] = acct_orgs['mjaccttypcd'].map(MACRO_TYPE_MAPPING)

In [ ]:
summary_df = acct_orgs.pivot_table(
    index='taxrptfororgnbr',
    columns='Macro Account Type',
    aggfunc={
        'Net Balance':'sum',
        'acctnbr':'nunique'
    },
    fill_value=0
)

In [ ]:
summary_df.columns = ['_'.join(col) for col in summary_df.columns]

In [ ]:
summary_df = summary_df.reset_index()

In [ ]:
summary_df['taxrptfororgnbr'] = summary_df['taxrptfororgnbr'].astype(int).astype(str)

In [ ]:
summary_df

In [ ]:
summary_df.info()

In [ ]:
summary_df = summary_df.rename(columns={
    'taxrptfororgnbr':'taxrptfororgnbr',
    'Net Balance_Deposit':'Deposit Balance',
    'Net Balance_Loan':'Loan Balance',
    'acctnbr_Deposit':'Unique Deposit Accounts',
    'acctnbr_Loan':'Unique Loan Accounts'
}).copy()

In [ ]:
summary_df

In [ ]:
# Get other entity details
entity_details = acct_orgs.groupby('taxrptfororgnbr').agg(
    primaryownercity=('primaryownercity', 'first'),
    primaryownerstate=('primaryownerstate','first'),
    branchname=('branchname','first'),
    earliest_opendate=('contractdate','min')
).reset_index()

In [ ]:
entity_details

In [ ]:
entity_details['taxrptfororgnbr'] = entity_details['taxrptfororgnbr'].astype(int).astype(str)

In [ ]:
# Merge two acct summary tables
summary_df = pd.merge(entity_details, summary_df, on='taxrptfororgnbr', how='inner')

In [ ]:
non_profits['orgnbr'] = non_profits['orgnbr'].astype(str)

In [ ]:
merged_df = pd.merge(non_profits, summary_df, left_on='orgnbr', right_on='taxrptfororgnbr', how='inner')

In [ ]:
merged_df = merged_df.drop(columns=['taxrptfororgnbr']).copy()

In [ ]:
merged_df['Total Accounts'] = merged_df['Unique Deposit Accounts'] + merged_df['Unique Loan Accounts']

In [ ]:
merged_df = merged_df.sort_values(by='Total Accounts', ascending=False).copy()

In [ ]:
merged_df

In [ ]:
# Append taxid
vieworgtaxid = org_data['vieworgtaxid'].copy()

In [ ]:
vieworgtaxid

In [ ]:
assert vieworgtaxid['orgnbr'].is_unique, "Duplicates"

In [ ]:
vieworgtaxid = vieworgtaxid[['orgnbr','taxid']].copy()

In [ ]:
vieworgtaxid['orgnbr'] = vieworgtaxid['orgnbr'].astype(str)

In [ ]:
merged_df = pd.merge(merged_df, vieworgtaxid, on='orgnbr', how='left')

In [ ]:
merged_df.info()

In [ ]:
merged_df

In [ ]:
genesis_data.info()

In [ ]:
genesis_data

In [ ]:
genesis_data['tax_id_clean'] = genesis_data['Tax ID'].replace("-","")

In [ ]:

dedupe_list = [{'df': genesis_data, 'field': 'tax_id_clean'}]
genesis_data = cdutils.deduplication.dedupe(dedupe_list)

In [ ]:
check_df = pd.merge(merged_df, genesis_data, left_on='taxid',right_on='tax_id_clean',how='inner')

In [ ]:
check_df

In [ ]:
# Will turn into formal pipeline after getting feedback from business line
